In [1]:
import os
import torch
import gc
import json
from huggingface_hub import login
import datasets
import transformers
from datasets import load_dataset, Dataset
from transformers import (AutoTokenizer,
                          AutoModelForCausalLM,
                          BitsAndBytesConfig,
                          TrainingArguments,
                          pipeline
                          )
from peft import (LoraConfig,
                  PeftConfig,
                  PeftModel,
                  get_peft_model,
                  prepare_model_for_kbit_training,
                  AutoPeftModelForCausalLM
                  )
import trl
from trl import (SFTTrainer,
                 setup_chat_format,
                 )
from multiprocessing import cpu_count
import pandas as pd

/home/talia/anaconda3/envs/finetune/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Could not find the bitsandbytes CUDA binary at PosixPath('/home/talia/anaconda3/envs/finetune/lib/python3.10/site-packages/bitsandbytes/libbitsandbytes_cuda113.so')
The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers, 8-bit multiplication, and GPU quantization are unavailable.


In [2]:
print(f"The PyTorch version is {torch.__version__}.")
print(f"Datasets version is {datasets.__version__}.")
print(f"Transformers version is {transformers.__version__}.")
print(f"TRL version is {trl.__version__}.")

The PyTorch version is 1.12.1.
Datasets version is 3.0.0.
Transformers version is 4.44.1.
TRL version is 0.11.0.


In [3]:

major_version, minor_version = torch.cuda.get_device_capability()
print(f"Cuda major version: {major_version}.\nCuda minor version: {minor_version}")
assert major_version >= 8, "Hardware not supported by Flash Attention."
print(f'CUDA capable device available - {torch.cuda.is_available()}')

Cuda major version: 8.
Cuda minor version: 6
CUDA capable device available - True


In [36]:
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = NVIDIA GeForce RTX 3080 Ti. Max memory = 11.749 GB.
10.406 GB of memory reserved.


# Data Explorer

In [36]:
main_df = pd.read_csv('data/text2cypher_claudeopus.csv')

FileNotFoundError: [Errno 2] No such file or directory: 'text2cypher_claudeopus.csv'

In [20]:
main_df.head()

,question,cypher,type,database,syntax_error,timeout,returns_results,false_schema
0,What are the top 5 movies with a runtime great...,MATCH (m:Movie)\nWHERE m.runtime > 120\nRETURN...,Simple Retrieval Queries,recommendations,False,False,True,NaN
1,List the first 3 directors born before 1950.,MATCH (d:Director)\nWHERE d.born < date('1950-...,Simple Retrieval Queries,recommendations,False,False,True,NaN
2,Which 5 users have rated more than 20 movies?,"MATCH (u:User)-[r:RATED]->(m:Movie)\nWITH u, c...",Simple Retrieval Queries,recommendations,False,False,True,NaN
3,Identify the top 5 actors who have acted in mo...,MATCH (a:Actor)-[:ACTED_IN]->(m:Movie)\nWITH a...,Simple Retrieval Queries,recommendations,False,False,True,NaN
4,What are the top 3 genres associated with movi...,MATCH (m:Movie)-[:IN_GENRE]->(g:Genre)\nWHERE ...,Simple Retrieval Queries,recommendations,False,False,True,NaN


In [21]:
schema_df = pd.read_csv('data/text2cypher_schemas.csv')

In [22]:
schema_df.head()

,database,schema,structured_schema
0,recommendations,Node properties:\n- **Movie**\n - `url`: STRI...,"{'node_props': {'Movie': [{'property': 'url', ..."
1,buzzoverflow,Node properties:\n- **Question**\n - `favorit...,{'node_props': {'Question': [{'property': 'fav...
2,bluesky,Node properties:\n- **User**\n - `label`: STR...,"{'node_props': {'User': [{'property': 'label',..."
3,companies,Node properties:\n- **Person**\n - `name`: ST...,{'node_props': {'Person': [{'property': 'name'...
4,fincen,Node properties:\n- **Country**\n - `location...,{'node_props': {'Country': [{'property': 'loca...


In [23]:
questions_df = pd.read_csv('data/text2cypher_questions.csv')

In [24]:
questions_df.head()

,question,type,database
0,What are the top 5 movies with a runtime great...,Simple Retrieval Queries,recommendations
1,List the first 3 directors born before 1950.,Simple Retrieval Queries,recommendations
2,Which 5 users have rated more than 20 movies?,Simple Retrieval Queries,recommendations
3,Identify the top 5 actors who have acted in mo...,Simple Retrieval Queries,recommendations
4,What are the top 3 genres associated with movi...,Simple Retrieval Queries,recommendations


# Model Definitions

In [2]:
data_path = 'data/'

In [3]:
trainer_with_repeats_file = 'parametric_trainer_with_repeats.json'

In [4]:
# Model from Hugging Face hub
model_id = "stabilityai/stable-code-instruct-3b"

# Fine-tuned model
model_path = "i_speak_cypher_3b"

In [5]:
with open(data_path+trainer_with_repeats_file, 'rb') as f:
	sampler = json.load(f)


In [6]:
sampler[123]

{'Prompt': 'Convert the following question into a Cypher query using the provided graph schema!',
 'Question': 'Fetch the Author nodes and extract their affiliation property!',
 'Schema': 'Graph schema: Relevant node labels and their properties (with datatypes) are:\nAuthor {affiliation: STRING}',
 'Cypher': 'MATCH (n:Author) RETURN n.affiliation'}

In [7]:
system_message = """
You are a text to Cypher query translator. {prompt}\n{schema}
"""

# Function to transform the data to conversational format {role:, content: }
def create_conversation(sample):
    return {
        "messages": [
            {"role": "system","content": system_message.format(prompt=sample["Prompt"], schema=sample["Schema"])},
            {"role": "user", "content": sample["Question"]},
            {"role": "assistant", "content": sample["Cypher"]}
        ]
    }

    


In [11]:
dataset = Dataset.from_list(sampler)
dataset = dataset.shuffle()

# Transform to required format
dataset = dataset.map(create_conversation,
                      remove_columns=dataset.features,
                      batched=False)


Map: 100%|██████████| 30116/30116 [00:02<00:00, 12065.56 examples/s]


In [12]:
#@title Split Data into Train and Test Sets
dataset = dataset.train_test_split(test_size=0.1, seed=23)

print(dataset)

DatasetDict({
    train: Dataset({
        features: ['messages'],
        num_rows: 27104
    })
    test: Dataset({
        features: ['messages'],
        num_rows: 3012
    })
})


In [13]:
dataset["train"][32]["messages"]

[{'content': "\nYou are a text to Cypher query translator. Convert the following question into a Cypher query using the provided graph schema!\nGraph schema: Relevant node labels and their properties (with datatypes) are:\nArticle {comments: STRING}\nAuthor {}\n\nRelevant relationships are:\n{'start': Article, 'type': WRITTEN_BY, 'end': Author }\n",
  'role': 'system'},
 {'content': 'Fetch comments of the Article that are connected to Author via WRITTEN_BY!',
  'role': 'user'},
 {'content': 'MATCH (n:Article) WHERE EXISTS { MATCH (n)-[:WRITTEN_BY]->(:Author) } RETURN n.comments AS comments',
  'role': 'assistant'}]

In [14]:
#@title Save the Dataset Splits
dataset["train"].to_json(data_path+"train_dataset.json", orient="records")
dataset["test"].to_json(data_path+"test_dataset.json", orient="records")

Creating json from Arrow format: 100%|██████████| 4/4 [00:00<00:00, 116.00ba/s]


2199514

In [15]:
tokenizer = AutoTokenizer.from_pretrained(model_id)

tokenizer.pad_token_id = tokenizer.eos_token_id
tokenizer.padding_side = 'right' # added to prevent warnings

# Set a maximum length
tokenizer.model_max_length = 2048

In [16]:

CHAT_TEMPLATE = "{% for message in messages %}\n{% if message['role'] == 'user' %}\n{{ '<|im_start|>user\n' + message['content'] + '<|im_end|>'+eos_token }}\n{% elif message['role'] == 'system' %}\n{{ '<|im_start|>system\n' + message['content'] + '<|im_end|>'+eos_token }}\n{% elif message['role'] == 'assistant' %}\n{{ '<|im_start|>assistant\n'  + message['content'] + '<|im_end|>'+eos_token }}\n{% endif %}\n{% if loop.last and add_generation_prompt %}\n{{ '<|im_start|>assistant' }}\n{% endif %}\n{% endfor %}"

tokenizer.chat_template = CHAT_TEMPLATE
     

In [17]:

train_dataset = load_dataset("json",
                       data_files=data_path+"train_dataset.json",
                       split="train")

Generating train split: 27104 examples [00:00, 832600.32 examples/s]


In [18]:
# Process the dataset function
def apply_chat_template(sample, tokenizer):
    messages = sample["messages"]

    # We add an empty system message if there is none
    if messages[0]["role"] != "system":
        messages.insert(0, {"role": "system", "content": ""})

    sample["text"] = tokenizer.apply_chat_template(messages,
                                                   tokenize=False)

    return sample

# Apply the chat template to the entire dataset
train_dataset = train_dataset.map(apply_chat_template,
                      fn_kwargs={"tokenizer": tokenizer},
                      remove_columns=train_dataset.features,
                      )
print(train_dataset)

Map: 100%|██████████| 27104/27104 [00:02<00:00, 11730.83 examples/s]

Dataset({
    features: ['text'],
    num_rows: 27104
})


In [19]:
print(train_dataset[123]["text"])

<|im_start|>system

You are a text to Cypher query translator. Convert the following question into a Cypher query using the provided graph schema!
Graph schema: Relevant node labels and their properties (with datatypes) are:
Author {affiliation: STRING}
Author {last_name: STRING}
<|im_end|><|endoftext|>
<|im_start|>user
Fetch the distinct values of the last_name from Author where either affiliation is unspecified or last_name is not null!<|im_end|><|endoftext|>
<|im_start|>assistant
MATCH (n:Author) WHERE n.affiliation = 'unspecified' OR n.last_name IS NOT NULL RETURN DISTINCT n.last_name AS last_name<|im_end|><|endoftext|>



In [20]:
device_map = {"": torch.cuda.current_device()} if torch.cuda.is_available() else None

In [72]:
device_map

{'': 0}

In [21]:
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map=device_map, #device_map,
    torch_dtype=torch.float16, #bfloat16, # change to float16 if using non-Ampere GPU
)


Loading checkpoint shards: 100%|██████████| 2/2 [00:02<00:00,  1.14s/it]


In [22]:
print(model)

StableLmForCausalLM(
  (model): StableLmModel(
    (embed_tokens): Embedding(50304, 2560)
    (layers): ModuleList(
      (0): StableLmDecoderLayer(
        (self_attn): StableLmAttention(
          (q_proj): Linear(in_features=2560, out_features=2560, bias=False)
          (k_proj): Linear(in_features=2560, out_features=2560, bias=False)
          (v_proj): Linear(in_features=2560, out_features=2560, bias=False)
          (o_proj): Linear(in_features=2560, out_features=2560, bias=False)
          (attention_dropout): Dropout(p=0.0, inplace=False)
          (rotary_emb): StableLmRotaryEmbedding()
        )
        (mlp): StableLmMLP(
          (gate_proj): Linear(in_features=2560, out_features=6912, bias=False)
          (up_proj): Linear(in_features=2560, out_features=6912, bias=False)
          (down_proj): Linear(in_features=6912, out_features=2560, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LayerNorm((2560,), eps=1e-05, elementwise_affine=True)
    

In [23]:
peft_config = LoraConfig(
        lora_alpha=128,
        lora_dropout=0.05,
        r=256,
        bias="none",
        target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "c_fc", "c_proj"], # use all linear layers
        task_type="CAUSAL_LM",
)
     

In [24]:

# Adapted from  Phil Schmid blogpost
args = TrainingArguments(
    output_dir=model_path,                  # directory to save the model and repository id
    num_train_epochs=1,
    per_device_train_batch_size=1,               # number of training epochs, use 3 at most
    gradient_accumulation_steps=2,
    gradient_checkpointing=True,            # use gradient checkpointing to save memory, use in distributed training
    gradient_checkpointing_kwargs={"use_reentrant": False}, # needed if gradient checkpoint is used
    optim="adamw_torch",              # use fused adamw optimizer
    logging_steps=10,                      # number of steps between two logs
    save_strategy="epoch",                  # save checkpoint every epoch
    learning_rate=2e-4,                     # learning rate, based on QLoRA paper
    tf32=True,                              # use tf32 precision for better performance
    max_grad_norm=1.0,                      # max gradient norm based on QLoRA paper
    warmup_steps=20,
    save_steps=100,                      # warmup ratio based on QLoRA paper
    lr_scheduler_type="cosine",           # use constant learning rate scheduler
)

In [25]:
trainer = SFTTrainer(
    model=model,
    args=args,
    train_dataset=train_dataset,
    dataset_text_field="text",
    peft_config=peft_config,
    max_seq_length=tokenizer.model_max_length,
    tokenizer=tokenizer,
    dataset_kwargs={
        "add_special_tokens": False,  # the template adds the special tokens
        "append_concat_token": False, # no need to add additional separator token
    }
)

/home/talia/anaconda3/envs/finetune/lib/python3.10/site-packages/huggingface_hub/utils/_deprecation.py:100: FutureWarning: Deprecated argument(s) used in '__init__': dataset_text_field, max_seq_length, dataset_kwargs. Will not be supported from version '1.0.0'.

Deprecated positional argument(s) used in SFTTrainer, please use the SFTConfig to set these arguments instead.
  warnings.warn(message, FutureWarning)
/home/talia/anaconda3/envs/finetune/lib/python3.10/site-packages/trl/trainer/sft_trainer.py:283: UserWarning: You passed a `max_seq_length` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(
/home/talia/anaconda3/envs/finetune/lib/python3.10/site-packages/trl/trainer/sft_trainer.py:321: UserWarning: You passed a `dataset_text_field` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(
/home/talia/anaconda3/envs/finetune/lib/python3.10/site-packages/trl/trainer/sft_train

In [26]:
trainer.train()

  0%|          | 10/13552 [00:03<1:14:49,  3.02it/s]

{'loss': 3.4745, 'grad_norm': 2.2865517139434814, 'learning_rate': 0.0001, 'epoch': 0.0}


  0%|          | 20/13552 [00:07<1:15:28,  2.99it/s]

{'loss': 1.4653, 'grad_norm': 1.4454466104507446, 'learning_rate': 0.0002, 'epoch': 0.0}


  0%|          | 30/13552 [00:10<1:12:26,  3.11it/s]

...

100%|█████████▉| 13550/13552 [1:18:16<00:00,  3.07it/s]

{'loss': 0.1328, 'grad_norm': 0.07617275416851044, 'learning_rate': 1.077967717311168e-11, 'epoch': 1.0}


100%|██████████| 13552/13552 [1:18:20<00:00,  2.88it/s]

{'train_runtime': 4700.5594, 'train_samples_per_second': 5.766, 'train_steps_per_second': 2.883, 'train_loss': 0.17071948282271096, 'epoch': 1.0}


TrainOutput(global_step=13552, training_loss=0.17071948282271096, metrics={'train_runtime': 4700.5594, 'train_samples_per_second': 5.766, 'train_steps_per_second': 2.883, 'total_flos': 8.403717176027136e+16, 'train_loss': 0.17071948282271096, 'epoch': 1.0})

In [27]:
trainer.save_model()

In [28]:

del trainer
del model
del tokenizer

gc.collect()
torch.cuda.empty_cache()

In [5]:
model = AutoPeftModelForCausalLM.from_pretrained(
  'i_speak_cypher_3b',
  device_map="auto",  # use auto for inference
  torch_dtype=torch.float16
)

tokenizer = AutoTokenizer.from_pretrained('i_speak_cypher_3b')
#model.resize_token_embeddings(len(tokenizer))

# Text generation pipeline
pipe = pipeline("text-generation", model=model, tokenizer=tokenizer)
     

Loading checkpoint shards: 100%|██████████| 2/2 [00:01<00:00,  1.26it/s]
The model 'PeftModelForCausalLM' is not supported for text-generation. Supported models are ['BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'LlamaForCausalLM', 'CodeGenForCausalLM', 'CohereForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'DbrxForCausalLM', 'ElectraForCausalLM', 'ErnieForCausalLM', 'FalconForCausalLM', 'FuyuForCausalLM', 'GemmaForCausalLM', 'Gemma2ForCausalLM', 'GitForCausalLM', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoForCausalLM', 'GPTNeoXForCausalLM', 'GPTNeoXJapaneseForCausalLM', 'GPTJForCausalLM', 'JambaForCausalLM', 'JetMoeForCausalLM', 'LlamaForCausalLM', 'MambaForCausalLM', 'Mamba2ForCausalLM', 'MarianForCausalLM', 'MBartForCausalLM', 'MegaForCaus

In [6]:
test_dataset = load_dataset("json", data_files=data_path+"test_dataset.json", split="train")

In [12]:
prompt = pipe.tokenizer.apply_chat_template(test_dataset[102]["messages"][:2],
                                            tokenize=False,
                                            add_generation_prompt=True)
print(prompt)

<|im_start|>system

You are a text to Cypher query translator. Convert the following question into a Cypher query using the provided graph schema!
Graph schema: Relevant node labels and their properties (with datatypes) are:
Article {article_id: INTEGER}
DOI {}

Relevant relationships are:
{'start': Article, 'type': HAS_DOI, 'end': DOI }
<|im_end|><|endoftext|>
<|im_start|>user
Search for the article_id values from 20 Article that are linked to DOI via HAS_DOI and return article_id along with the respective DOI counts!<|im_end|><|endoftext|>
<|im_start|>assistant



In [13]:
outputs = pipe(prompt,
              max_new_tokens=256,
              do_sample=False,
              temperature=0.1,
              top_k=50,
              top_p=0.1
              )

print(f"Question: {test_dataset[102]['messages'][1]['content']}")
print(f"Correct Cypher: {test_dataset[102]['messages'][2]['content']}")
print(f"Generated Cypher: {outputs[0]['generated_text'][len(prompt):-10].strip()}")

/home/talia/anaconda3/envs/finetune/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:567: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.1` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/home/talia/anaconda3/envs/finetune/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:572: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.1` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


Question: Search for the article_id values from 20 Article that are linked to DOI via HAS_DOI and return article_id along with the respective DOI counts!
Correct Cypher: MATCH (n:Article) -[:HAS_DOI]->(m:DOI) WITH DISTINCT n, m RETURN n.article_id AS article_id, count(m) AS count LIMIT 20
Generated Cypher: MATCH (n:Article) -[:HAS_DOI]->(m:DOI) WITH DISTINCT n, m RETURN n.article_id AS article_id, count(m) AS count LIMIT 20<


In [10]:
#@title Test on a Subset of Samples
from tqdm import tqdm

# Compare the generated text with provided Cypher statement

def evaluate(sample):
    prompt = pipe.tokenizer.apply_chat_template(sample["messages"][:2],
                                                tokenize=False,
                                                add_generation_prompt=True)
    outputs = pipe(prompt,
                   max_new_tokens=256,
                   do_sample=True,
                   temperature=0.7,
                   top_k=50,
                   top_p=0.95
                   )

    predicted_answer = outputs[0]['generated_text'][len(prompt):-10].strip() # remove end generation token

    if predicted_answer[:-1] == sample["messages"][2]["content"]:
        return 1
    else:
        return 0

success_rate = []
number_of_eval_samples = 100

# Iterate over sample dataset and predict
for s in tqdm(test_dataset.shuffle().select(range(number_of_eval_samples))):
    success_rate.append(evaluate(s))

# compute accuracy
accuracy = sum(success_rate)/len(success_rate)

print(f"Accuracy: {accuracy*100:.2f}%")

100%|██████████| 100/100 [03:21<00:00,  2.01s/it]

Accuracy: 71.00%


In [13]:
!python -m pip install numpy==1.24.1

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 34.4 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: numpy
    Found existing installation: numpy 2.1.1
    Uninstalling numpy-2.1.1:
      Successfully uninstalled numpy-2.1.1
